In [1]:
import tkinter as tk
from tkinter import ttk, messagebox

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from io import BytesIO

import random

import onnxruntime as ort

In [2]:
class GridWorld:
    def __init__(self, rows, cols, start, terminal_states, walls, rewards, towers,tower_penalty, tower_timer, 
                 num_towers=1, retour=True, step_cost=-0.01, broken=False, tower_destroy_reward=0.0):
        self.rows = rows
        self.cols = cols
        self.start = start
        self.terminal_states = terminal_states
        self.walls = walls
        self.rewards = rewards
        self.step_cost = step_cost
        self.action_space = 5
        self.retour = retour
        
        # Nombre de tours (limité à 3 maximum)
        self.num_towers = min(3, max(1, num_towers)) 
        
        self.broken = broken
        self.tower_destroy_reward = tower_destroy_reward 
        
        self.initial_tower_position = towers
        self.tower_positions = towers
        self.tower_cells = []
        self.tower_destroyed = [False] * self.num_towers 
        self.tower_penalty = tower_penalty  
        self.tower_timer = tower_timer  
        
        self.reset()
        
        self.position_history = []
        self.max_history_length = 30  
        self.steps_count = 0  

    def get_tower_cells(self, center):
        """Retourne les 9 cellules de la tour 3x3 centrée à la position donnée"""
        cells = []
        i, j = center
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = i + di, j + dj
                if 0 <= ni < self.rows and 0 <= nj < self.cols:
                    cells.append((ni, nj))
        return cells
        
    def reset(self):
        self.agent_pos = self.start
        self.position_history = [self.start]
        self.steps_count = 0
        
        self.tower_positions = self.initial_tower_position
        self.tower_cells = [self.get_tower_cells(pos) for pos in self.tower_positions]
        self.tower_destroyed = [False] * self.num_towers
        
        return self.get_state_representation()

    def is_terminal(self, state_pos):
        return tuple(state_pos) in self.terminal_states

    def get_state_representation(self):
        """
        Représentation d'état améliorée avec un voisinage 15x15 et information sur les tours
        """
        pos_i, pos_j = self.agent_pos
        goal_i, goal_j = self.terminal_states[0]
        # Position normalisée
        norm_pos_i = pos_i / self.rows
        norm_pos_j = pos_j / self.cols
        # Distance à l'objectif (normalisée)
        dist_i = (goal_i - pos_i) / self.rows
        dist_j = (goal_j - pos_j) / self.cols
        # Distance euclidienne normalisée
        euclidean_dist = np.sqrt((goal_i - pos_i)**2 + (goal_j - pos_j)**2) / np.sqrt(self.rows**2 + self.cols**2)
        # Distance de Manhattan normalisée
        manhattan_dist = (abs(goal_i - pos_i) + abs(goal_j - pos_j)) / (self.rows + self.cols)
        
        # Construire une carte des murs dans un voisinage 15x15
        wall_map = np.zeros(224)  # 224 positions autour de l'agent (15x15 sans la position centrale)
        idx = 0
        for di in range(-7, 8):
            for dj in range(-7, 8):
                if di == 0 and dj == 0:  # Ignorer la position de l'agent
                    continue
                ni, nj = pos_i + di, pos_j + dj
                # Si c'est un mur ou hors limites
                if (ni, nj) in self.walls or not (0 <= ni < self.rows and 0 <= nj < self.cols):
                    wall_map[idx] = 1
                idx += 1
                
        recent_visits = self.position_history[-15:] 
        revisit_count = min(0.75, recent_visits.count(self.agent_pos) / 7.0) 
        
        start_i, start_j = self.start
        start_dist = np.sqrt((start_i - pos_i)**2 + (start_j - pos_j)**2) / np.sqrt(self.rows**2 + self.cols**2)
        
        tower_states = []
        for idx, (tower_pos, tower_cells, destroyed) in enumerate(zip(self.tower_positions, self.tower_cells, self.tower_destroyed)):
            tower_exists = 1.0 if (self.steps_count < self.tower_timer and not destroyed) else 0.0
            in_tower = 1.0 if self.agent_pos in tower_cells and tower_exists else 0.0
            
            tower_dist = 0.0
            tower_dir_i, tower_dir_j = 0.0, 0.0
            if tower_exists:
                tower_i, tower_j = tower_pos
                tower_dist = np.sqrt((tower_i - pos_i)**2 + (tower_j - pos_j)**2) / np.sqrt(self.rows**2 + self.cols**2)
                # Direction vers la tour (vecteur normalisé)
                tower_dir_i = (tower_i - pos_i) / self.rows
                tower_dir_j = (tower_j - pos_j) / self.cols
            
            # Temps restant avant disparition de la tour (normalisé)
            tower_time_left = max(0, (self.tower_timer - self.steps_count) / self.tower_timer) if tower_exists else 0.0
            
            # Indicateur si la tour a été détruite
            tower_destroyed_flag = 1.0 if destroyed else 0.0
            
            # Ajouter les informations de cette tour à la liste
            tower_states.extend([
                tower_exists,         # La tour existe-t-elle encore? (1)
                in_tower,             # L'agent est-il dans la tour? (1)
                tower_dist,           # Distance à la tour (1)
                tower_time_left,      # Temps restant avant disparition (1)
                tower_dir_i,          # Direction vers la tour (i) (1)
                tower_dir_j,          # Direction vers la tour (j) (1)
                tower_destroyed_flag  # Indicateur si détruite (1)
            ])
        
        # Compléter avec des zéros si moins de 3 tours (pour garder un vecteur de taille fixe)
        while len(tower_states) < 21:  # 7 infos par tour * 3 tours max = 21
            tower_states.extend([0.0] * 7)
        
        state = np.array([
            norm_pos_i, norm_pos_j,                # Position normalisée (2)
            dist_i, dist_j,                        # Direction vers l'objectif (2)
            manhattan_dist, euclidean_dist,        # Distances à l'objectif (2)
            start_dist,                            # Distance au départ (1)
            revisit_count,                         # Indicateur de boucles (1)
            float(self.num_towers)                 # Nombre de tours dans l'environnement (1)
        ] + tower_states + wall_map.tolist(), dtype=np.float32)   # Infos tours (21) + Carte des murs 15x15 (224)
        
        return state

    def step(self, action):
        action_effects = {
            0: (-1, 0),  # up
            1: (1, 0),   # down
            2: (0, -1),  # left
            3: (0, 1),   # right
            4: (0, 0)    # wait
        }

        if self.is_terminal(self.agent_pos):
            return self.get_state_representation(), 0, True

        next_pos = (self.agent_pos[0] + action_effects[action][0],
                    self.agent_pos[1] + action_effects[action][1])

        # Vérifier si le prochain état est un mur ou hors limites
        if next_pos in self.walls or not (0 <= next_pos[0] < self.rows and 0 <= next_pos[1] < self.cols):
            next_pos = self.agent_pos  # L'agent reste sur place
            reward = -0.2  # Pénalité modérée pour avoir heurté un mur
        else:
            self.agent_pos = next_pos
            reward = self.rewards.get(self.agent_pos, self.step_cost)
            
            # Vérifier si l'agent est entré dans une tour active
            for idx, (tower_cells, destroyed) in enumerate(zip(self.tower_cells, self.tower_destroyed)):
                tower_active = (self.steps_count < self.tower_timer and not destroyed)
                if tower_active and self.agent_pos in tower_cells:
                    if self.broken:
                        # En mode broken, l'agent détruit la tour et reçoit une récompense
                        if not destroyed:
                            self.tower_destroyed[idx] = True
                            reward += self.tower_destroy_reward
                            self.tower_cells[idx] = []  #
                            try:
                                self.tower_positions.pop(idx) # Supprimer la position de la tour (par index)
                            except:
                                print("Déjà cassé")
                            
                            if idx < len(self.tower_cells):
                                self.tower_cells.pop(idx)
                            if idx < len(self.tower_destroyed):
                                self.tower_destroyed.pop(idx)
                            
                            break 
                    else:
                        # En mode normal, l'agent reçoit une pénalité
                        reward += self.tower_penalty
                        if self.retour:
                            self.agent_pos = self.start
    
        if action == 4:  # wait
            reward -= 0.005  # Pénalité pour l'attente   
            
        # Incrémenter le compteur de pas
        self.steps_count += 1
        
        if self.agent_pos in self.position_history[-10:]:
            visits = self.position_history[-10:].count(self.agent_pos)
            reward -= visits * 0.08  # Pénalité pour dissuader les boucles
        
        # Mettre à jour l'historique des positions
        self.position_history.append(self.agent_pos)
        if len(self.position_history) > self.max_history_length:
            self.position_history.pop(0)
        
        # Bonus basé sur le progrès vers l'objectif (récompense de shaping)
        goal_i, goal_j = self.terminal_states[0]
        curr_dist = abs(self.agent_pos[0] - goal_i) + abs(self.agent_pos[1] - goal_j)
        
        if len(self.position_history) > 1:
            prev_pos = self.position_history[-2]
            prev_dist = abs(prev_pos[0] - goal_i) + abs(prev_pos[1] - goal_j)
            if curr_dist < prev_dist:
                reward += 0.05  # Bonus plus important pour se rapprocher
            elif curr_dist > prev_dist:
                reward -= 0.02  # Légère pénalité pour s'éloigner
            elif curr_dist == prev_dist:
                reward += 0.02 # Encourage l'agent à attendre
        
        done = self.is_terminal(self.agent_pos)
        if done:
            reward = self.rewards.get(self.agent_pos, 0)
            
        return self.get_state_representation(), reward, done

    def render(self):
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_xlim(0, self.cols)
        ax.set_ylim(0, self.rows)
        ax.set_xticks(np.arange(0, self.cols + 1, 1))
        ax.set_yticks(np.arange(0, self.rows + 1, 1))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(color="black", linestyle="-", linewidth=1)

        for wall in self.walls:
            rect = patches.Rectangle((wall[1], self.rows - wall[0] - 1), 1, 1, facecolor="black")
            ax.add_patch(rect)

        for terminal in self.terminal_states:
            rect = patches.Rectangle((terminal[1], self.rows - terminal[0] - 1), 1, 1, facecolor="green")
            ax.add_patch(rect)
            
        tower_colors = ["orange", "purple", "brown"]  # Différentes couleurs pour les tours
        for idx, (tower_cells, destroyed) in enumerate(zip(self.tower_cells, self.tower_destroyed)):
            if self.steps_count < self.tower_timer and not destroyed:
                color = tower_colors[idx % len(tower_colors)]
                for cell in tower_cells:
                    rect = patches.Rectangle((cell[1], self.rows - cell[0] - 1), 1, 1, 
                                          facecolor=color, alpha=0.6)
                    ax.add_patch(rect)

        rect = patches.Rectangle((self.agent_pos[1], self.rows - self.agent_pos[0] - 1), 1, 1, facecolor="blue")
        ax.add_patch(rect)
        
        # Visualiser l'historique des positions (avec gradient de couleur)
        for i, pos in enumerate(self.position_history[:-1]):
            alpha = 0.1 + 0.4 * i / len(self.position_history)
            rect = patches.Rectangle((pos[1], self.rows - pos[0] - 1), 1, 1, 
                                  facecolor="lightblue", alpha=min(0.3, alpha))
            ax.add_patch(rect)

        destroyed_count = sum(self.tower_destroyed)
        if destroyed_count > 0:
            ax.text(0.5, 0.02, f"{destroyed_count}/{self.num_towers} tours détruites!", 
                   transform=ax.transAxes, ha='center', 
                   bbox=dict(facecolor='red', alpha=0.8))
        
        if self.steps_count < self.tower_timer:
            time_left = self.tower_timer - self.steps_count
            ax.text(0.5, 0.05, f"Tours: {time_left} pas restants", transform=ax.transAxes, 
                   ha='center', bbox=dict(facecolor='white', alpha=0.8))

        plt.title("GridWorld")

        buf = BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight')
        buf.seek(0)
        image = Image.open(buf)
        plt.close(fig)

        return image

In [3]:
class GridWorldInterface:
    def __init__(self, root):
        self.root = root
        self.root.title("GridWorld Interface")
        self.root.geometry("700x600")
        
        # Configuration par défaut
        self.rows = 20
        self.cols = 9
        self.cell_size = 25
        self.max_walls = 60
        self.max_towers = 3
        self.walls = []
        self.towers = []  # Liste des positions des tours
        self.start = (0, self.cols//2)
        self.terminal_states = [(self.rows-1, self.cols//2)]
        self.selected_agent = "dqn_tower.onnx"  # Agent par défaut
        self.tower_penalty = -5
        self.tower_timer = 20
        self.retour = True
        self.broken = False
        self.tower_destroy_reward = 0.0
        self.num_agents = 3  # Nombre d'agents par défaut
        
        # Variables pour le suivi des clics
        self.current_action = "wall"  # "wall" ou "tower"
        self.is_mouse_down = False # Pour suivre si le bouton de la souris est enfoncé
        
        # Frame principal
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Panneau de configuration (gauche)
        self.config_frame = ttk.LabelFrame(self.main_frame, text="Configuration")
        self.config_frame.pack(side="left", fill="y", padx=5, pady=5)
        
        # Sélecteur d'agent
        ttk.Label(self.config_frame, text="Sélectionner l'agent:").pack(anchor="w", padx=5, pady=5)
        self.agent_var = tk.StringVar(value=self.selected_agent)
        agent_combo = ttk.Combobox(self.config_frame, textvariable=self.agent_var, 
                                          values=["dqn_base.onnx", "dqn_tower.onnx","dqn_break.onnx"])
        agent_combo.pack(padx=5, pady=5, fill="x")
        agent_combo.bind("<<ComboboxSelected>>", self.update_agent)
        
        # Sélecteur du nombre d'agents
        ttk.Label(self.config_frame, text="Nombre d'agents:").pack(anchor="w", padx=5, pady=5)
        self.num_agents_var = tk.StringVar(value=str(self.num_agents))
        num_agents_combo = ttk.Combobox(self.config_frame, textvariable=self.num_agents_var, 
                                          values=["1", "2", "3", "5", "10"])
        num_agents_combo.pack(padx=5, pady=5, fill="x")
        num_agents_combo.bind("<<ComboboxSelected>>", self.update_num_agents)
        
        # Mode d'édition
        ttk.Label(self.config_frame, text="Mode d'édition:").pack(anchor="w", padx=5, pady=5)
        self.edit_mode_frame = ttk.Frame(self.config_frame)
        self.edit_mode_frame.pack(padx=5, pady=5, fill="x")
        
        self.edit_mode = tk.StringVar(value="wall")
        ttk.Radiobutton(self.edit_mode_frame, text="Placer mur", variable=self.edit_mode, 
                                          value="wall", command=self.set_edit_mode).pack(side="left")
        ttk.Radiobutton(self.edit_mode_frame, text="Placer tour", variable=self.edit_mode, 
                                          value="tower", command=self.set_edit_mode).pack(side="left")
        
        # Compteurs
        self.counter_frame = ttk.Frame(self.config_frame)
        self.counter_frame.pack(padx=5, pady=5, fill="x")
        
        self.wall_counter_var = tk.StringVar(value=f"Murs: 0/{self.max_walls}")
        self.tower_counter_var = tk.StringVar(value=f"Tours: 0/{self.max_towers}")
        
        ttk.Label(self.counter_frame, textvariable=self.wall_counter_var).pack(anchor="w")
        ttk.Label(self.counter_frame, textvariable=self.tower_counter_var).pack(anchor="w")
        
        # Boutons d'action
        self.buttons_frame = ttk.Frame(self.config_frame)
        self.buttons_frame.pack(padx=5, pady=10, fill="x")
        
        ttk.Button(self.buttons_frame, text="Effacer tout", command=self.clear_grid).pack(fill="x", pady=2)
        ttk.Button(self.buttons_frame, text="Lancer simulation", command=self.run_simulation).pack(fill="x", pady=2)
        
        # Canvas pour la grille (centre)
        self.canvas_frame = ttk.LabelFrame(self.main_frame, text="Grille")
        self.canvas_frame.pack(side="right", fill="both", expand=True, padx=5, pady=5)
        
        # Calculer la taille du canvas basée sur la taille de la grille
        canvas_width = self.cols * self.cell_size + 1
        canvas_height = self.rows * self.cell_size + 1
        
        self.canvas = tk.Canvas(self.canvas_frame, width=canvas_width, height=canvas_height, 
                                            bg="white", borderwidth=0, highlightthickness=1)
        self.canvas.pack(padx=5, pady=5, expand=True)
        
        # Lier les événements du canvas
        self.canvas.bind("<Button-1>", self.start_action)  # Détecter le clic initial
        self.canvas.bind("<ButtonRelease-1>", self.stop_action)  # Détecter le relâchement
        self.canvas.bind("<B1-Motion>", self.perform_action)    # Détecter le mouvement avec le bouton enfoncé
        self.canvas.bind("<Button-3>", self.delete_object)  # Right click to delete
        
        # Dessiner la grille initiale
        self.draw_grid()
    
    def update_num_agents(self, event):
        try:
            self.num_agents = int(self.num_agents_var.get())
        except ValueError:
            self.num_agents = 3  # Valeur par défaut si la conversion échoue
            self.num_agents_var.set(str(self.num_agents))
    
    def set_edit_mode(self):
        self.current_action = self.edit_mode.get()
    
    def update_agent(self, event):
        self.selected_agent = self.agent_var.get()
    
    def draw_grid(self):
        self.canvas.delete("all")
        
        # Dessiner les lignes de la grille
        for i in range(self.rows + 1):
            y = i * self.cell_size
            self.canvas.create_line(0, y, self.cols * self.cell_size, y, fill="gray")
        
        for j in range(self.cols + 1):
            x = j * self.cell_size
            self.canvas.create_line(x, 0, x, self.rows * self.cell_size, fill="gray")
        
        # Dessiner le point de départ (en bleu)
        start_row, start_col = self.start
        x1 = start_col * self.cell_size
        y1 = start_row * self.cell_size
        self.canvas.create_rectangle(x1, y1, x1 + self.cell_size, y1 + self.cell_size, fill="blue", tags="start")
        
        # Dessiner le point d'arrivée (en vert)
        end_row, end_col = self.terminal_states[0]
        x1 = end_col * self.cell_size
        y1 = end_row * self.cell_size
        self.canvas.create_rectangle(x1, y1, x1 + self.cell_size, y1 + self.cell_size, fill="green", tags="end")
        
        # Dessiner les murs existants
        for wall in self.walls:
            row, col = wall
            x1 = col * self.cell_size
            y1 = row * self.cell_size
            self.canvas.create_rectangle(x1, y1, x1 + self.cell_size, y1 + self.cell_size, fill="black", tags="wall")
        
        # Dessiner les tours existantes
        for tower_center in self.towers:
            self.draw_tower(tower_center)
    
    def draw_tower(self, center):
        center_row, center_col = center
        
        # Calculer les 9 cellules de la tour
        tower_cells = []
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = center_row + di, center_col + dj
                if 0 <= ni < self.rows and 0 <= nj < self.cols:
                    tower_cells.append((ni, nj))
        
        # Dessiner chaque cellule de la tour
        for cell in tower_cells:
            row, col = cell
            x1 = col * self.cell_size
            y1 = row * self.cell_size
            self.canvas.create_rectangle(x1, y1, x1 + self.cell_size, y1 + self.cell_size, 
                                            fill="orange", outline="black", tags="tower")
    
    def start_action(self, event):
        self.is_mouse_down = True
        self.perform_action(event) # Pour que le placement se fasse aussi au premier clic
    
    def stop_action(self, event):
        self.is_mouse_down = False

    def perform_action(self, event):
        if not self.is_mouse_down:
            return  # Ne rien faire si le bouton n'est pas enfoncé
        
        # Calculer la position de la cellule cliquée
        col = event.x // self.cell_size
        row = event.y // self.cell_size
        
        # Vérifier si la position est valide
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return
        
        cell_pos = (row, col)
        
        # Ne pas modifier les cellules de départ et d'arrivée
        if cell_pos == self.start or cell_pos in self.terminal_states:
            return
        
        if self.current_action == "wall":
            self.handle_wall_click(cell_pos)
        elif self.current_action == "tower":
            self.handle_tower_click(cell_pos)
        
        # Mettre à jour les compteurs
        self.wall_counter_var.set(f"Murs: {len(self.walls)}/{self.max_walls}")
        self.tower_counter_var.set(f"Tours: {len(self.towers)}/{self.max_towers}")
        
        # Redessiner la grille
        self.draw_grid()
    
    def handle_wall_click(self, cell_pos):
        # Si le clic est sur une cellule de tour, ne rien faire
        for tower_center in self.towers:
            tower_cells = self.get_tower_cells(tower_center)
            if cell_pos in tower_cells:
                return
        
        # Ajouter/enlever un mur
        if cell_pos not in self.walls:
            self.walls.append(cell_pos)
    
    def handle_tower_click(self, cell_pos):
        # Vérifier si le clic est sur une tour existante
        for tower_center in self.towers[:]:
            tower_cells = self.get_tower_cells(tower_center)
            if cell_pos in tower_cells:
                self.towers.remove(tower_center)
                # Supprimer la tour du canvas
                self.canvas.delete("tower")
                # Redessiner toutes les tours restantes
                for t in self.towers:
                    self.draw_tower(t)
                return
        
        # Vérifier si on peut ajouter une nouvelle tour
        if len(self.towers) < self.max_towers:
            # Vérifier que la tour ne chevauche pas les murs, le départ ou l'arrivée
            tower_cells = self.get_tower_cells(cell_pos)
            valid = True
            for cell in tower_cells:
                if cell in self.walls or cell == self.start or cell in self.terminal_states:
                    valid = False
                    break
            
            # Vérifier que la tour ne chevauche pas une autre tour
            for tower_center in self.towers:
                existing_cells = self.get_tower_cells(tower_center)
                if any(cell in existing_cells for cell in tower_cells):
                    valid = False
                    break
            
            if valid:
                self.towers.append(cell_pos)
                self.draw_tower(cell_pos)
    
    def get_tower_cells(self, center):
        center_row, center_col = center
        cells = []
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = center_row + di, center_col + dj
                if 0 <= ni < self.rows and 0 <= nj < self.cols:
                    cells.append((ni, nj))
        return cells
    
    def clear_grid(self):
        self.walls = []
        self.towers = []
        self.draw_grid()
        self.wall_counter_var.set(f"Murs: 0/{self.max_walls}")
        self.tower_counter_var.set(f"Tours: 0/{self.max_towers}")
    
    def is_valid_maze(self):
        queue = [self.start]
        visited = {self.start}
        
        while queue:
            current = queue.pop(0)
            if current in self.terminal_states:
                return True
                
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = current[0] + dx, current[1] + dy
                neighbor = (nx, ny)
                
                if (0 <= nx < self.rows and 0 <= ny < self.cols and 
                    neighbor not in self.walls and 
                    neighbor not in visited):
                    visited.add(neighbor)
                    queue.append(neighbor)
        
        return False
    
    def run_simulation(self):
        # Vérifier si le labyrinthe est valide
        if not self.is_valid_maze():
            messagebox.showerror("Erreur", "Aucun chemin possible entre le départ et l'arrivée !")
            return
        
        # Mettre à jour le nombre d'agents selon la valeur actuelle
        try:
            self.num_agents = int(self.num_agents_var.get())
        except ValueError:
            self.num_agents = 3
            self.num_agents_var.set(str(self.num_agents))
        
        # Paramètres pour l'environnement
        rewards = {self.terminal_states[0]: 10.0}
        step_cost = -0.01

        match self.selected_agent:
            case 'dqn_base.onnx':
                self.tower_penalty = -0.0
                self.tower_timer = 30
                self.retour = True
                self.broken = False
                self.tower_destroy_reward = 0.0
            case 'dqn_tower.onnx':
                self.tower_penalty = -5.0
                self.tower_timer = 30
                self.retour = True
                self.broken = False
                self.tower_destroy_reward = 0.0
            case _ :
                self.tower_penalty = -0.0
                self.tower_timer = 30
                self.retour = False
                self.broken = True
                self.tower_destroy_reward = 5.0
        
        # Créer les environnements pour chaque agent
        environments = []
        
        # Créer des positions de départ légèrement différentes pour chaque agent
        start_positions = []
        start_row, start_col = self.start
        
        # Générer des positions de départ variées autour du point de départ
        for i in range(self.num_agents):
            if i == 0:
                # Le premier agent démarre à la position par défaut
                start_positions.append(self.start)
            else:
                # Les autres agents démarrent à proximité avec un petit décalage
                offset_row = 0
                offset_col = i % (self.cols - 1) - (self.cols - 1) // 2
                
                # S'assurer que la position est valide
                new_row = max(0, min(self.rows - 1, start_row + offset_row))
                new_col = max(0, min(self.cols - 1, start_col + offset_col))
                new_pos = (new_row, new_col)
                
                # Vérifier que la position n'est pas un mur
                if new_pos in self.walls:
                    # Si c'est un mur, utiliser la position par défaut
                    new_pos = self.start
                
                start_positions.append(new_pos)
        
        # Créer un environnement pour chaque agent avec sa position de départ
        for i in range(self.num_agents):
            env = GridWorld(
                self.rows, 
                self.cols, 
                start_positions[i], 
                self.terminal_states, 
                self.walls, 
                rewards, 
                self.towers,
                self.tower_penalty,
                self.tower_timer,
                len(self.towers),
                self.retour,
                step_cost,
                self.broken,
                self.tower_destroy_reward
            )
            
            # Définir les positions des tours
            tower_cells = []
            for tower_center in self.towers:
                env.tower_position = tower_center
                env.tower_cells = env.get_tower_cells(tower_center)
                tower_cells.extend(env.tower_cells)
            
            environments.append(env)
        
        # Charger l'agent ONNX avec une exploration aléatoire différente pour chaque instance
        agents = [ONNXAgent(self.selected_agent, exploration_rate=0.05 + i * 0.05) for i in range(self.num_agents)]
        
        # Exécuter la simulation multi-agents
        self.run_multi_agent_simulation(environments, agents)
    
    def run_multi_agent_simulation(self, environments, agents):
        # Créer une nouvelle fenêtre pour la simulation
        sim_window = tk.Toplevel(self.root)
        sim_window.title("Simulation Multi-Agents GridWorld")
        sim_window.geometry("700x800")
        
        # Frame pour les informations
        info_frame = ttk.Frame(sim_window)
        info_frame.pack(fill="x", expand=True, padx=10, pady=5)
        
        # Créer un frame pour contenir les info_frame avec scrollbar
        scroll_frame = ttk.Frame(sim_window)
        scroll_frame.pack(fill="x", padx=10, pady=5)
        
        # Ajouter une scrollbar horizontale
        scrollbar = ttk.Scrollbar(scroll_frame, orient="horizontal")
        scrollbar.pack(fill="x", side="bottom")
        
        # Canvas pour contenir les infos avec scrollbar
        info_canvas = tk.Canvas(scroll_frame, height=50, xscrollcommand=scrollbar.set)
        info_canvas.pack(fill="x", expand=True)
        
        # Configurer la scrollbar
        scrollbar.config(command=info_canvas.xview)
        
        # Frame interne pour les infos d'agents
        inner_frame = ttk.Frame(info_canvas)
        info_canvas.create_window((0, 0), window=inner_frame, anchor="nw")
        
        # Labels pour afficher les informations sur chaque agent
        agent_info_labels = []
        colors = self.generate_agent_colors(self.num_agents)
        
        for i in range(self.num_agents):
            agent_frame = ttk.Frame(inner_frame)
            agent_frame.pack(side="left", padx=10)
            
            color_label = ttk.Label(agent_frame, text="■", foreground=colors[i], font=("Arial", 12, "bold"))
            color_label.pack(side="left")
            
            info_label = ttk.Label(agent_frame, text=f"Agent {i+1}: Pas=0, Récompense=0.0")
            info_label.pack(side="left")
            agent_info_labels.append(info_label)
        
        # Mettre à jour la région défilable
        inner_frame.update_idletasks()
        info_canvas.config(scrollregion=info_canvas.bbox("all"))
        
        # Canvas pour afficher l'animation
        sim_canvas = tk.Canvas(sim_window, width=700, height=600, bg="white")
        sim_canvas.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Bouton pour fermer la simulation
        ttk.Button(sim_window, text="Fermer", command=lambda: [self.draw_grid(), sim_window.destroy()]).pack(pady=10)
        
        # Initialiser les variables pour chaque agent
        states = [env.reset() for env in environments]
        dones = [False] * self.num_agents
        steps = [0] * self.num_agents
        total_rewards = [0.0] * self.num_agents
        max_steps = self.rows * self.cols * 2
        
        # Pour l'affichage
        cell_size = min(600 // max(self.rows, self.cols), 30)
        
        # Dictionnaire pour suivre les chemins de chaque agent
        paths = [[] for _ in range(self.num_agents)]
        
        def update_simulation():
            nonlocal steps, states, dones, total_rewards, paths
            
            # Vérifier si tous les agents ont terminé ou atteint le nombre max de pas
            all_done = all(dones) or all(step >= max_steps for step in steps)
            
            if not all_done:
                # Faire avancer chaque agent non terminé
                for i in range(self.num_agents):
                    if not dones[i] and steps[i] < max_steps:
                        # Obtenir l'action de l'agent correspondant
                        action = agents[i].select_action(states[i])
                        
                        # Effectuer l'action
                        next_state, reward, done = environments[i].step(action)
                        # Mettre à jour les variables
                        states[i] = next_state
                        total_rewards[i] += reward
                        steps[i] += 1
                        dones[i] = done
                        
                        # Enregistrer la position actuelle pour le chemin
                        paths[i].append(environments[i].agent_pos)
                
                # Afficher l'état actuel des environnements
                self.draw_multi_agent_environment(sim_canvas, environments, cell_size, colors, paths)
                
                # Mettre à jour les informations pour chaque agent
                for i in range(self.num_agents):
                    status = "✓" if dones[i] else ""
                    if steps[i] >= max_steps and not dones[i]:
                        status = "✗"
                    agent_info_labels[i].config(
                        text=f"Agent {i+1}: Pas={steps[i]}, Récompense={total_rewards[i]:.2f} {status}"
                    )
                
                # Planifier la prochaine mise à jour
                sim_window.after(100, update_simulation)
            else:
                # Simulation terminée, afficher les résultats finaux
                final_text = "Simulation terminée\n"
                
                # Trier les résultats par meilleure récompense
                results = []
                for i in range(self.num_agents):
                    success = "Objectif atteint !" if dones[i] else "Échec"
                    results.append((i, steps[i], total_rewards[i], success))
                
                # Trier par récompense puis par nombre de pas (si même récompense)
                results.sort(key=lambda x: (-x[2], x[1]))
                
                for i, step, reward, success in results:
                    final_text += f"Agent {i+1}: {step} pas, {reward:.2f} de récompense - {success}\n"
                
                # Afficher les résultats dans une boîte de dialogue
                messagebox.showinfo("Résultats de la simulation", final_text)
        
        # Dessiner l'environnement initial
        self.draw_multi_agent_environment(sim_canvas, environments, cell_size, colors, paths)
        
        # Démarrer la simulation
        sim_window.after(500, update_simulation)
    
    def generate_agent_colors(self, num_agents):
        """Génère des couleurs distinctes pour chaque agent"""
        # Liste de couleurs prédéfinies pour les agents
        predefined_colors = [
            "#FF0000",  # Rouge
            "#0000FF",  # Bleu
            "#00AA00",  # Vert
            "#FF00FF",  # Magenta
            "#00FFFF",  # Cyan
            "#FFA500",  # Orange
            "#800080",  # Violet
            "#008080",  # Teal
            "#800000",  # Marron
            "#000080",  # Bleu marine
        ]
        
        # S'il n'y a pas assez de couleurs prédéfinies, en générer aléatoirement
        if num_agents <= len(predefined_colors):
            return predefined_colors[:num_agents]
        else:
            colors = predefined_colors.copy()
            for _ in range(num_agents - len(predefined_colors)):
                # Générer une couleur aléatoire en évitant celles qui sont trop similaires aux existantes
                while True:
                    r = random.randint(0, 255)
                    g = random.randint(0, 255)
                    b = random.randint(0, 255)
                    
                    # Convertir en format hexadécimal
                    new_color = f"#{r:02x}{g:02x}{b:02x}"
                    
                    # Vérifier que la couleur est différente des autres
                    if new_color not in colors:
                        colors.append(new_color)
                        break
            return colors
    
    def draw_multi_agent_environment(self, canvas, environments, cell_size, colors, paths):
        canvas.delete("all")
        
        # Utilisez le premier environnement comme référence pour la structure
        env_ref = environments[0]

        # Dessiner les lignes de la grille
        for i in range(env_ref.rows + 1):
            y = i * cell_size
            canvas.create_line(0, y, env_ref.cols * cell_size, y, fill="gray")

        for j in range(env_ref.cols + 1):
            x = j * cell_size
            canvas.create_line(x, 0, x, env_ref.rows * cell_size, fill="gray")

        # Dessiner les états terminaux (en vert)
        for terminal in env_ref.terminal_states:
            row, col = terminal
            x1 = col * cell_size
            y1 = row * cell_size
            canvas.create_rectangle(x1, y1, x1 + cell_size, y1 + cell_size, fill="green")

        # Dessiner les murs (en noir)
        for wall in env_ref.walls:
            row, col = wall
            x1 = col * cell_size
            y1 = row * cell_size
            canvas.create_rectangle(x1, y1, x1 + cell_size, y1 + cell_size, fill="black")

        # Dessiner les tours si elles sont encore actives
        if env_ref.tower_positions:
            for t, tower in enumerate(env_ref.tower_positions):
                if env_ref.steps_count < env_ref.tower_timer:
                    for cell in env_ref.tower_cells[t]:
                        row, col = cell
                        x1 = col * cell_size
                        y1 = row * cell_size
                        canvas.create_rectangle(x1, y1, x1 + cell_size, y1 + cell_size, fill="orange", stipple="gray50")

        # Dessiner les chemins parcourus pour chaque agent
        for i, path in enumerate(paths):
            for pos in path[:-1]:  # Tous sauf la position actuelle
                row, col = pos
                x1 = col * cell_size
                y1 = row * cell_size
                
                # Dessiner le chemin avec une transparence (stipple)
                canvas.create_rectangle(
                    x1, y1, x1 + cell_size, y1 + cell_size, 
                    fill=colors[i], stipple="gray25", outline=""
                )
        
        # Dessiner les agents avec leur couleur respective
        for i, env in enumerate(environments):
            row, col = env.agent_pos
            x1 = col * cell_size
            y1 = row * cell_size
            
            # Dessiner chaque agent avec sa couleur
            canvas.create_rectangle(
                x1, y1, x1 + cell_size, y1 + cell_size, 
                fill=colors[i], outline="black", width=2
            )
            
            # Afficher le numéro de l'agent
            canvas.create_text(
                x1 + cell_size/2, y1 + cell_size/2, 
                text=str(i+1), fill="white", font=("Arial", 10, "bold")
            )

        # Afficher le temps restant pour les tours
        if env_ref.tower_positions:
            for i, tower in enumerate(env_ref.tower_positions):
                if env_ref.steps_count < env_ref.tower_timer:
                    time_left = env_ref.tower_timer - env_ref.steps_count
                    canvas.create_text(
                        env_ref.cols * cell_size // 2, 10 + i * 20,
                        text=f"Tour {i+1}: {time_left} pas restants",
                        font=("Arial", 10), fill="black"
                    )
    
    def delete_object(self, event):
        # Calculer la position de la cellule cliquée
        col = event.x // self.cell_size
        row = event.y // self.cell_size
        cell_pos = (row, col)
        
        # Vérifier si la position est valide
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return
        
        # Supprimer un mur si présent
        if cell_pos in self.walls:
            self.walls.remove(cell_pos)
            self.wall_counter_var.set(f"Murs: {len(self.walls)}/{self.max_walls}")
        
        # Supprimer une tour si présente
        for tower_center in self.towers[:]:
            tower_cells = self.get_tower_cells(tower_center)
            if cell_pos in tower_cells:
                self.towers.remove(tower_center)
                self.tower_counter_var.set(f"Tours: {len(self.towers)}/{self.max_towers}")
                break # Important: sortir de la boucle après avoir supprimé une tour
        
        # Redessiner la grille pour refléter les changements
        self.draw_grid()


class ONNXAgent:
    def __init__(self, model_path, exploration_rate=0.0):
        self.session = ort.InferenceSession(f'RL_2/{model_path}')
        self.input_name = self.session.get_inputs()[0].name
        self.output_name = self.session.get_outputs()[0].name
        self.exploration_rate = exploration_rate  # Taux d'exploration (epsilon)
    
    def select_action(self, state):
        # Exploration aléatoire
        if random.random() < self.exploration_rate:
            return random.randint(0, 4)  
        
        # Exploitation: utiliser le modèle
        input_data = np.array(state, dtype=np.float32).reshape(1, -1)
        output = self.session.run([self.output_name], {self.input_name: input_data})
        q_values = output[0][0]
        
        # Ajouter un petit bruit aléatoire aux Q-values pour éviter les égalités
        noise = np.random.normal(0, 0.01, q_values.shape)
        q_values += noise
        
        return np.argmax(q_values)

if __name__ == "__main__":
    import tkinter as tk
    root = tk.Tk()
    app = GridWorldInterface(root)
    root.mainloop()


Déjà cassé
